In [1]:
from langchain_community.document_loaders import PyPDFLoader


def extract_text_from_pdf(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    return ' '.join([doc.page_content for doc in documents])

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_text_into_chunks(text: str, chunk_size: int = 1000, chunk_overlap: int = 200):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return text_splitter.create_documents([text])

In [50]:
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

embeddings = OllamaEmbeddings(model="nomic-embed-text:latest", num_gpu=8)

def create_vector_store(chunks, persist_dir="chroma_db"):
    return Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=persist_dir  # directory on disk where DB is stored
    )

In [51]:
import os
import hashlib

from langchain.schema import Document
from tqdm import tqdm

def deduplicate_documents(docs):
    """Remove duplicate documents based on text hash."""
    seen = set()
    unique_docs = []
    for doc in docs:
        # hash page_content
        hash_val = hashlib.md5(doc.page_content.strip().encode("utf-8")).hexdigest()
        if hash_val not in seen:
            seen.add(hash_val)
            unique_docs.append(doc)
    return unique_docs

folder_path = 'vector_database_pdfs'  # Replace with the actual path to your folder

# Get all entries in the directory
all_entries = os.listdir(folder_path)

chunks = []

# Iterate through each entry
for entry_name in tqdm(all_entries):
    full_path = os.path.join(folder_path, entry_name)
    
    pdf_text = extract_text_from_pdf(full_path)
    # Check if the entry is a file (not a directory)
    if os.path.isfile(full_path):
        chunks += split_text_into_chunks(pdf_text, chunk_size=1000, chunk_overlap=200)
        
        
chunks = deduplicate_documents(chunks)
print("Deduplicated Documents")

# Create or update persistent DB
persist_dir = "chroma_db"
vector_store = create_vector_store(chunks, persist_dir=persist_dir)
vector_store._collection.count()

100%|██████████| 7/7 [00:30<00:00,  4.30s/it]


Deduplicated Documents


6540

In [86]:
# title = "AITA for telling my bestfriend I understood why his fiancee is so disappointed that he proposed to her at the gym ?"

# post = """I (27f) was excited when my bestfriend (27m) told me he had proposed to his girlfriend (29f). He said he had the proposal on video. I was so confused when the video started out in woman's gym. When I saw his girlfriend on an exercise machine in the video, I had a bad feeling. She looked so shocked and she said yes. She only looked happy for 2 minutes and the rest of the time she gave an insincere smile.

# My bestfriend expressed frustration that his fiancee had confessed she wasn't happy with the proposal after he asked her why she looked so sad. He was venting to me and asked me how I feel if a guy proposed to me while I was at the gym. I guess he really expected me to agree with him, but I said I would hate it. I said I don't want to be proposed to when I'm sweaty and stinky at the gym. I basically explained to him that I understood why she was disappointed. My bestfriend called me shallow and a bad friend. Am I the asshole ?
# """

# title = "AITA For requesting to be paid back"

# post = """So this all happened last Wednesday (8/20) and I'm genuinely so annoyed.

# So, I booked an Airbnb for my birthday this year (it was back in February) for me and my friends to go and enjoy. Well, two of the friends (a married couple who genuinely shouldn't be married to each other) leave and don't spend the entire weekend with me, thus missing my actual birthday, leaving just me and one other friend. The Airbnb was like $1500 for six days (2/20 through 2/25). They were supposed to pay me back, as we had discussed over dinner the day I booked/paid for it.

# (I don't worry about the friend who stayed with me through the entire time; they're good for the money.)

# My (former) best friend had been sending me pictures of rings that she'd been wanting to get, even though she complained she doesn't have money right now, despite her father in law giving them FIFTEEN THOUSAND to pay off her husband's Jeep Gladiator and her dad giving her $500 "just because".

# I told her last Wednesday (again, when all this went down) that I saw things differently right now because I'm broke as hell and don't think she should get ANOTHER ring set (she has three already) if she's complaining about money.

# Cut to me being blocked everywhere and my roommate (friend who stayed with me through the entirety of my birthday weekend) asked for the whole story and hasn't gotten it yet - and it's been over a week - meaning that there isn't more to it.

# Guess I lost a friend AND $750 🫠

# OH, and she got her hair done from a really dark brown to a strawberry blonde color (easily $300+) and her nose repierced (another $100+), and yet, she "doesn't have money to pay me back".

# My family thinks I should take her to small claims court with all the proof (credit card statements, pictures of her getting the aforementioned hair and nose piercing, etc) to help bolster my potential case.

# AITA?
# """

title = "AITA - Asked a Service Dog owner to move at the Movies."

post = """I 32F was able to get some solo time away from the husband and kids and I wanted to see Freakier Friday. Just wanted to have a drink, eat some popcorn, and watch the sequel to one of my favorite childhood movies.

I'm severely allergic to animals. Dogs, cats, and horses. Not anaphylaxis bad, but it's pretty severe. Due to that, I avoid animals like the plague and purposely do not go to dog friendly places. I can generally tell when a dog is near before I even see it.

I got to the theater early, got my drink, popcorn and took my seat. I got a top row seat. Ten minutes later, I see a golden retriever (I think) wearing a service dog vest with its owner (20'sF) and it sits 2 rows in front of me, a few seats over. It lays down and appears well behaved. But well behaved dogs are still an issue for me.

I gently inquired if it was a service animal. The owner just looks at me and glares at me for a second. In my head, i'm just like "Fuck". I speak up again and tell the lady I'm severely allergic and can you at least sit across the theater. She firmly refuses to move and mentions that she's tired of people bothering her about her dog. I was like, I was here first and am settled in. I also asked her to please be reasonable. She says her dog is allowed to be here and basically says too bad for you. I gather my things and angrily told her thanks for ruining the movie for me. She sarcastically says "bye" to me. I just left, didn't see the movie. I just went home. AITA here?

ETA: I had the aisle seat of the top row. I wanted her to move to the other end of the row she was in or down to the front."""

ollama_model = 'llama4:scout'

In [87]:
from ollama import chat
from pydantic import BaseModel

class Topics(BaseModel):
  psychology: list[str]
  ethical_philosophy: list[str]


response = chat(
  messages=[
    {
      'role': 'user',
      'content': f'''
      Given this post from r/AITA give me a list of relevant topics from fields such as psychology and ethical philosophy that would help OP.
      Title: {title}
      Post: {post}
      ''',
      "options": {
        "thinking": False
      }
    }
  ],
  model=ollama_model,
  format=Topics.model_json_schema(),
)

topics = Topics.model_validate_json(response.message.content)
print(topics)

psychology=['emotional regulation', 'conflict resolution', 'assertiveness', 'boundary setting', 'anxiety management', 'trauma-informed care', 'emotional intelligence'] ethical_philosophy=['utilitarianism', 'deontology', 'virtue ethics', 'care ethics', 'moral relativism', 'social contract theory']


In [88]:
topics_list = topics.psychology + topics.ethical_philosophy

topics_list

['emotional regulation',
 'conflict resolution',
 'assertiveness',
 'boundary setting',
 'anxiety management',
 'trauma-informed care',
 'emotional intelligence',
 'utilitarianism',
 'deontology',
 'virtue ethics',
 'care ethics',
 'moral relativism',
 'social contract theory']

In [89]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

retrieved_docs = []

for topic in tqdm(topics_list):
    retrieved_docs += vector_store.similarity_search(topic)

len(retrieved_docs)

docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)


100%|██████████| 13/13 [00:00<00:00, 46.24it/s]


In [90]:
print(len(retrieved_docs))

52


In [91]:
print(docs_content)

as used by Feldman Barrett (2017a), appears to portra y a more positiv e view than “ emotion regulation”, 
because the expression of “mastering” embraces the agency of those involved in emotional experiences. The 
idea of emotions ne eding to be regulate d/controlled are based on the the triune brain model. This model 
proposes that emotions are loc alised to the limbic s ystem, which is thought to be more ancient than our 
“highly developed” rational neo-cortex and which needs to be managed by the latter. As noted in Chapter 
1.2, the loc alisation of emotions to one p articular area, as w ell as the limbic s ystem as such, ha ve be en 
proven wrong by neuroscience research. Nevertheless, emotion regulation is the main term used in relevant 
literature, so we will use this term throughout this topic. 
Please watch the following video [6:09] by Sam Hardy (use d with permission) to learn more about what 
emotion regulation means:

literature, so we will use this term throughout this top

In [92]:

template = """Summarize the content involving psychology and ethical philosphy below in a set of bullet points:
    {docs_content}
"""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model=ollama_model, options={"thinking": False})

chain = prompt | model

summarized_docs_content = chain.invoke({"docs_content": docs_content})

print(summarized_docs_content)

It seems like you've provided a large text on ethics, morality, and social contract theory. I'll try to break it down and provide a clear summary.

**Main topics:**

1. **Moral Relativism**: The idea that moral judgments are relative to cultural or personal perspectives.
2. **Social Contract Theory**: The idea that moral rules are based on agreements between individuals to cooperate and maintain social order.

**Key points:**

1. **Moral Relativism**:
	* Critics argue that relativism fails to provide a universal moral standard.
	* Some argue that relativism is based on a flawed assumption that moral judgments are solely based on cultural or personal perspectives.
2. **Social Contract Theory**:
	* Social contract theory posits that moral rules are based on agreements between individuals to cooperate and maintain social order.
	* Theorists like Thomas Hobbes and John Rawls argue that individuals, as rational and self-interested agents, agree to follow moral rules to achieve cooperation a

In [93]:


template = """

You are an r/AITA commenter, who is commenting on the given post. However, unlike most r/AITA users, you have expertise in psychology, ethical philosophy, and communication. Use this knowlege to decide whether or not OP is the asshole. Then tell OP what to do next.

Also, remember this notation, as it is used in the r/AITA subreddit. Bring up concepts from the context in your response, and connect them to the post. Add a TLDR at the end of your response. At some point in your response, you must explicitly put one of the following in your response:

NTA: Not the asshole

YTA: You're the asshole

NAH: No assholes here

ESH: Everyone's an Asshole

Context:
{context}

Title: {title}

Post: {post}

"""

prompt = ChatPromptTemplate.from_template(template)

docs_content = summarized_docs_content

model = OllamaLLM(model=ollama_model, options={"thinking": True})

chain = prompt | model

response = chain.invoke({
    "title": title, 
    "post": post, 
    "context": summarized_docs_content
})

print(response)

The situation presented involves a conflict between two individuals with competing needs and rights. On one hand, there's the OP who has severe allergies and requires a certain level of accommodation to feel safe and healthy. On the other hand, there's the service dog owner who has a legitimate right to be in the theater with her service animal.

From a moral and ethical standpoint, it's essential to consider the principles of respect, empathy, and understanding. The OP's request for the service dog owner to move is not unreasonable, given the severity of their allergies and the potential risks associated with being near the dog.

However, the service dog owner's refusal to move and her assertion that she has a right to be in the theater with her service animal are also valid points. It's crucial to acknowledge that service animals play a vital role in assisting individuals with disabilities, and their presence is protected by laws and regulations.

In this situation, the OP's decision

# Sources

All textbooks come from UMN's open textbook library

*Philosophical Ethicsa guidebook for beginners* by, George W. Matthews

*Communication in the Real World - An Introduction to Communication Studies* LibreTexts

*Introduction to Community Psychology* by Leonard A. Jason, Olya Glantsman, Jack F. O'Brien, and Kaitlyn N. Ramian (Editors)

*Introduction to Philosophy: Ethics* by, Frank Aragbonfoh Abumere; Douglas Giles; Ya-Yun (Sherry) Kao; Michael Klenk; Joseph Kranak; Kathryn MacKay; Jeffrey Morgan; Paul Rezkalla; George Matthews (Book Editor); and Christina Hendricks (Series Editor)

*Introduction to Ethics: An Open Educational Resource* by Noah Levin Ph.D.

*Neuroscience, psychology and conflict management* by Judith Rafferty 

*Metaethics from a  First Person Standpoint: An Introduction to Moral Philosophy* by Catherine Wilson

